In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
credits= pd.read_csv("E:/DataScienceProject/archive/tmdb_5000_credits.csv")

In [ ]:
movies=pd.read_csv("E:/DataScienceProject/tmdb_5000_movies.csv")


movies.head(5)

In [ ]:
movies

In [ ]:
credits

In [ ]:
credits.shape

In [ ]:
movies.shape

In [ ]:
movies= movies.merge( credits, on ='title')

In [ ]:
movies.shape
movies.isnull().sum()

In [ ]:
#will recoomendate on 7 criteria 
#genres
#id
#keyword
#title
#overview
#cast
#crew
movies.columns


In [ ]:
movies= movies[['genres','id', 'keywords','title', 'overview','cast', 'crew']]

In [ ]:
movies

In [ ]:
movies.shape

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def gen(obj):
    L= []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L    


In [ ]:
movies

In [ ]:
movies

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
def key(obj):
    L= []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [ ]:
movies['keywords'] = movies['keywords'].apply(key)

In [ ]:
def cast(obj):
    L= []
    counter = 0 
    for i in ast.literal_eval(obj):
        if counter!= 3:
            L.append(i['name'])
            counter =counter+1
       
    return L  

In [ ]:
movies['cast']= movies['cast'].apply(cast)

In [ ]:
def cr(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']== 'Director':
            L.append(i['name'])
    return L

In [ ]:
movies['crew']= movies['crew'].apply(cr)

In [ ]:
movies['genres']= movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords']= movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast']= movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew']= movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [ ]:
movies['overview']= movies['overview'].str.split()

In [ ]:
movies['tags']= movies['genres']+ movies['cast']+ movies['crew']+ movies['overview']+ movies['keywords']

In [ ]:
movies= movies[['id','title','tags']]

In [ ]:
def join_lists(x):
    if isinstance(x, list):
        return ' '.join(x)
    return x if x is not None else ''

In [ ]:
movies['tags'] = movies['tags'].apply(join_lists)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cs =CountVectorizer(max_features= 5000, stop_words='english')

In [ ]:
movies['tags'] = movies['tags'].fillna('')

In [ ]:
vector = cs.fit_transform(movies['tags']).toarray()
vector

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    index = movies[movies['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(movies.iloc[i[0]].title)

In [ ]:
recommend('Spider-Man')